In [2]:
import os
import random
from PIL import Image

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F

import logging

In [3]:
num_classes = 6

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 72 * 72 , 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,num_classes)

    def forward(self,x):
      x=self.pool(F.relu(self.conv1(x)))
      x=self.pool(F.relu(self.conv2(x)))
      x=x.view(-1,self.num_flat_features(x))
      x=F.relu(self.fc1(x))
      x=F.relu(self.fc2(x))
      return self.fc3(x)

    def num_flat_features(self,x):
      size=x.size()[1:]
      num_features=1
      for s in size:
          num_features *=s

      return num_features


In [4]:
logging.basicConfig(level=logging.INFO,
                    format='%(message)s',
                    handlers=[logging.FileHandler("log.txt"),
                              logging.StreamHandler()])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #쿠다주세요쿠다주세요

dirs = ['applejack', 'fluttershy', 'pinkie pie', 'rainbow dash', 'rarity', 'twilight sparkle']

def is_valid_image(image_path):
    try:
        img = Image.open(image_path)
        img.verify()
        return True
    except:
        return False
    
def resize_image(image_path, new_width=300, new_height=300):
    try:
        img = Image.open(image_path)
        img = img.convert("RGB")
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        return img
    except Exception as e:
        print(f"Error occurred with file {image_path}: {e}")
        return None


transformations=transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


#-------------------------------------모델 선택----------------------------------------------
model = SimpleCNN()
model_path = "save/model3.pth"                      
model.load_state_dict(torch.load(model_path))
model.to(device)
print("Model loaded")

model.eval()

#-------------------------------------폴더 선택----------------------------------------------
test_dir = "mlp"

class_count = {name: 0 for name in ['applejack', 'fluttershy', 'pinkie pie', 'rainbow dash', 'rarity', 'twilight sparkle']}

for file in os.listdir(test_dir):
    image_path = os.path.join(test_dir, file)

    if is_valid_image(image_path):
        img = resize_image(image_path)
        if img is not None:
            img_tensor = transformations(img).unsqueeze(0).to(device)  # Transform image and add batch dimension
            
            output=model(img_tensor)
            _, predicted=torch.max(output.data,1)

            logging.info(f"예측: {dirs[predicted.item()]}, 이미지: {file}")
            print(f"예측: {dirs[predicted.item()]}, 이미지: {file}")

            class_count[dirs[predicted.item()]] += 1  # Increment count of predicted class

logging.info(class_count)
print(class_count)


C:\Users\sando\AppData\Local\Temp\ipykernel_23048\1629858931.py:26: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((new_width, new_height), Image.ANTIALIAS)
예측: twilight sparkle, 이미지: pony (1).gif
예측: twilight sparkle, 이미지: pony (1).jpeg


Model loaded
예측: twilight sparkle, 이미지: pony (1).gif
예측: twilight sparkle, 이미지: pony (1).jpeg


예측: rarity, 이미지: pony (1).jpg
예측: pinkie pie, 이미지: pony (1).png


예측: rarity, 이미지: pony (1).jpg


예측: rainbow dash, 이미지: pony (1).webp
예측: pinkie pie, 이미지: pony (10).gif
예측: pinkie pie, 이미지: pony (10).jpeg


...

예측: twilight sparkle, 이미지: pony (98).png
예측: twilight sparkle, 이미지: pony (980).gif
예측: pinkie pie, 이미지: pony (981).gif
예측: applejack, 이미지: pony (982).gif
예측: twilight sparkle, 이미지: pony (983).gif


예측: rarity, 이미지: pony (984).gif
예측: applejack, 이미지: pony (985).gif
예측: applejack, 이미지: pony (986).gif
예측: applejack, 이미지: pony (987).gif
예측: twilight sparkle, 이미지: pony (988).gif
예측: twilight sparkle, 이미지: pony (99).gif
예측: twilight sparkle, 이미지: pony (99).jpg


예측: rarity, 이미지: pony (984).gif
예측: applejack, 이미지: pony (985).gif
예측: applejack, 이미지: pony (986).gif
예측: applejack, 이미지: pony (987).gif
예측: twilight sparkle, 이미지: pony (988).gif
예측: twilight sparkle, 이미지: pony (99).gif


예측: twilight sparkle, 이미지: pony (99).png
{'applejack': 278, 'fluttershy': 458, 'pinkie pie': 340, 'rainbow dash': 333, 'rarity': 473, 'twilight sparkle': 924}


예측: twilight sparkle, 이미지: pony (99).jpg
예측: twilight sparkle, 이미지: pony (99).png
{'applejack': 278, 'fluttershy': 458, 'pinkie pie': 340, 'rainbow dash': 333, 'rarity': 473, 'twilight sparkle': 924}
